In [1]:
import pandas as pd

In [4]:
import numpy as np
import scipy
import math
import random

In [5]:
import importlib

In [6]:
import sklearn
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

In [7]:
from gensim import models

C:\Users\HP\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
import Recommenders as Rd

In [9]:
importlib.reload(Rd)

<module 'Recommenders' from 'C:\\Users\\HP\\Anaconda3\\Scripts\\Natural-Language-Processing\\Recommender-systems\\cleaned-version\\Recommenders.py'>

In [23]:
import Recommenders_v1 as Rd1

import matplotlib.pyplot as plt

## Explore Interaction Events Data

### -----------------------------------------------------------------------------------------------------------------------------------
Contains logs of user interactions on shared articles. It can be joined to articles_shared.csv by contentId column.

The eventType values are:

    VIEW: The user has opened the article.
    LIKE: The user has liked the article.
    COMMENT CREATED: The user created a comment in the article.
    FOLLOW: The user chose to be notified on any new comment in the article.
    BOOKMARK: The user has bookmarked the article for easy return in the future.
### -----------------------------------------------------------------------------------------------------------------------------------

In [10]:
interactions_df = pd.read_csv('users_interactions.csv')

In [11]:
articles_df = pd.read_csv('shared_articles.csv')

### Data Muning

### -----------------------------------------------------------------------------------------------------------------------------------
    Convert Event from categories to event strength :
    we associate them with a weight or strength, assuming that, for example, a comment in an article indicates a higher interest
    of the user on the item than a like, or than a simple view
### -----------------------------------------------------------------------------------------------------------------------------------

In [12]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
# 

In [13]:
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [14]:

interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])

In [15]:
df_inter = interactions_df[['contentId','personId','eventStrength']]

In [16]:
df_acrticles = articles_df[['contentId','title','text']]

In [17]:
merged = pd.merge(df_inter, df_acrticles,how='inner', on=['contentId'])

### Number of Users and items 

In [19]:
users = merged['personId'].unique() ; print("# of users" , len(users))

# of users 1895


In [20]:
items = merged['contentId'].unique() ; print("# of items" , len(items))

# of items 2979


In [21]:
train_data, test_data = train_test_split(merged, test_size = 0.20, random_state=0)

In [30]:
user_id = users[5]

### Test the model Content_based_Recommendation with Doc2vec

##### ----- Build the recommender 

### Basic Steps of the recommender building : ----------------------------------------------------------------------------------------
    It has the same principle as tfidf_features recommender here we just use an other feature given by doc2vec
   
    1 - we train the doc2vec model from articles contents ( 'title' and 'text' ) we got a matrix of (n_articles , n_features = 1000)
    2 - for each item in user items list get its nearest neighbors in terms of cosine similarity (topn)
    3 - Compute the score of each collected item in the neighbors set by summing all gotten scores 
    4 - select k-top best items
### ------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
model = models.Doc2Vec.load("doc2vec_article_1000")

In [320]:
importlib.reload(Rd1)

<module 'Recommenders_v1' from 'C:\\Users\\HP\\Anaconda3\\Scripts\\Natural-Language-Processing\\Recommender-systems\\cleaned-version\\Recommenders_v1.py'>

In [321]:
doc_model = Rd1.content_relatedness_doc2vec_py()

In [322]:
doc_model.create(train_data, 'personId', 'contentId')

In [323]:
df =  doc_model.recommend(user_id,model,top = 30,topn = 40)

Start building user profile
Done!


In [324]:
df

,title,contentId,score
55,Fooling The Machine,8219310215587599928,27.286744
116,Google DeepMind researchers built a deep-learn...,-3751249597609107201,24.972333
10734,Deep learning software knows that a rose is a ...,5314107418764884230,23.611559
5399,NYT: Google To Announce Alexa Competitor Calle...,7168347449326627154,23.125300
9470,"Need Some AI? Yeah, There's a Marketplace for ...",-455124319026316634,22.408055
94,AI can learn from data without ever having acc...,-1173087925894083974,20.323239
8891,YouTube's New Messenger Means You'll Never Hav...,4563606877148407012,20.171472
8787,Facebook Messenger bots are here and they want...,-4615575698508618335,19.870518
5599,A new Facebook chatbot could help you find you...,2546657529517570876,19.754812
7554,"Here's what Viv looks like, the next generatio...",5979676385148714519,19.447297
